In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import string
!pip install -U scikit-learn scipy matplotlib

!pip install profanity

import nltk
nltk.download('all')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression


from sklearn.metrics import f1_score

Requirement already up-to-date: scikit-learn in /home/sam/.local/lib/python3.8/site-packages (1.2.0)
Requirement already up-to-date: scipy in /home/sam/.local/lib/python3.8/site-packages (1.9.3)
Requirement already up-to-date: matplotlib in /home/sam/.local/lib/python3.8/site-packages (3.6.2)


[nltk_data] Error loading all: <urlopen error [Errno 110] Connection
[nltk_data]     timed out>


In [2]:
main_df=pd.read_csv("/home/sam/Desktop/Semester_5/ML/project2/yahoo-troll-question-detection/train_df.csv")
test_df=pd.read_csv("/home/sam/Desktop/Semester_5/ML/project2/yahoo-troll-question-detection/test_df.csv")
sub_df=pd.read_csv("/home/sam/Desktop/Semester_5/ML/project2/yahoo-troll-question-detection/sample_submission.csv")

In [3]:
main_df_copy=main_df

In [4]:
main_df_copy_x=main_df_copy.drop(axis="columns",labels="target")
main_df_copy_y=main_df_copy["target"].astype(np.float64)

In [5]:
vectorizer = CountVectorizer(ngram_range=(1,3))
vectorizer.fit(main_df_copy_x['question_text'])
train_cv = vectorizer.transform(main_df_copy_x["question_text"])

In [6]:
test_cv=vectorizer.transform(test_df["question_text"].to_numpy())

### Calculating the profanity index for the question text

In [7]:
# def prof(column):
#     arr=np.zeros(column.size)
#     count=0
#     for i in column:
#         arr[count]=profanity.contains_profanity(i)
#         count+=1
#     return arr

In [8]:
# profanity_train=prof(main_df["question_text"])
# profanity_test=prof(test_df["question_text"])

In [9]:
# train_cv=scipy.sparse.hstack((train_cv,profanity_train.reshape(-1,1))).tocsr()
# test_cv=scipy.sparse.hstack((test_cv,profanity_test.reshape(-1,1))).tocsr()

In [10]:
model = LogisticRegression(max_iter=50000,class_weight={0:0.0843683326353982,1:0.39255923905226237})

In [11]:
model.fit(train_cv,main_df_copy_y)

LogisticRegression(class_weight={0: 0.0843683326353982, 1: 0.39255923905226237},
                   max_iter=50000)

In [12]:
pred_train=model.predict_proba(train_cv)[:,1]
# pred_train
pred_train=(pred_train>0.68).astype(np.int64)
f1_score(main_df_copy_y,pred_train)

0.8762351697015764

In [13]:
pred_test=model.predict_proba(test_cv)[:,1]
pred_test=(pred_test>0.42105959647081376).astype(np.int64)

In [14]:
# pred_test=model.predict(test_cv)
# pred_test=(pred_test>0.52).astype(np.int64)

In [15]:
test_lr_cv_pred=pd.DataFrame(pred_test)
datasets=pd.concat([sub_df['qid'],test_lr_cv_pred],axis=1)
datasets.columns=["qid","target"]

In [16]:
datasets["target"]=datasets["target"].astype(np.int64)

In [17]:
datasets.to_csv("./1,3_0.42105959647081376_0.0843683326353982_0.39255923905226237.csv",index=False)